## Introduction

In this notebook, you'll see a Microsoft Fabric data science workflow with an end-to-end example. The scenario is to build a model to predict whether bank customers would churn or not. The churn rate, also known as the rate of attrition refers to the rate at which bank customers stop doing business with the bank.

The main steps in this notebook are:

1. Install custom libraries
2. Load the data
3. Understand and process the data through exploratory data analysis and demonstrate the use of Fabric Data Wrangler feature
4. Train machine learning models using `Scikit-Learn`, and track experiments using MLflow and Fabric Autologging feature
5. Evaluate and save the final machine learning model

## Import libraries


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import coalesce,concat_ws
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import numpy as np
import pandas as pd
# Import the required libraries for model training
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix, recall_score, roc_auc_score, classification_report

StatementMeta(, e88e1b73-7973-4663-8141-0c848a616fce, 3, Finished, Available, Finished)

## Load the data

The dataset contains churn status of 10,000 customers along with 14 attributes

Out of the 10,000 customers, only 2037 customers (around 20%) have left the bank. Therefore, given the class imbalance ratio, it is recommended to generate synthetic data.

|"CustomerID"|"Surname"|"CreditScore"|"Geography"|"Gender"|"Age"|"Tenure"|"Balance"|"NumOfProducts"|"HasCrCard"|"IsActiveMember"|"EstimatedSalary"|"Exited"|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|15634602|Hargrave|619|France|Female|42|2|0.00|1|1|1|101348.88|1|
|15647311|Hill|608|Spain|Female|41|1|83807.86|1|0|1|112542.58|0|




In [2]:
# Initialize Spark session (if not already initialized)
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
df=spark.table("customer_churn")
df = df.toPandas()

StatementMeta(, e88e1b73-7973-4663-8141-0c848a616fce, 4, Finished, Available, Finished)

### Cleansed to Curated 

In [3]:
# Code generated by Data Wrangler for pandas DataFrame

def clean_data(df):
    # Replace missing values with 0 in columns: 'Balance', 'Tenure'
    df = df.fillna({'Balance': 0, 'Tenure': 0})
    # Drop duplicate rows in columns: 'RowNumber', 'CustomerId'
    df = df.drop_duplicates(subset=['RowNumber', 'CustomerId'])
    return df

df_clean_tr = clean_data(df.copy())

StatementMeta(, e88e1b73-7973-4663-8141-0c848a616fce, 21, Finished, Available, Finished)

### Save the clean data

In [4]:
table_name = "Curated_Lakehouse.df_clean"
# Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(df_clean_tr) 
sparkDF.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable(table_name)
print(f"Spark dataframe saved to delta table: {table_name}")

StatementMeta(, e88e1b73-7973-4663-8141-0c848a616fce, 22, Finished, Available, Finished)

Spark dataframe saved to delta table: Curated_Lakehouse.df_clean


## ML Task: Create, evaluate, and score a churn prediction model

## Feature Engineering

In [19]:
newdf=df_clean_tr.drop(columns=['RowNumber', 'CustomerId', 'Surname','IsDeleted'])
df_clean = pd.get_dummies(newdf, columns=['Geography', 'Gender'])

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 21, Finished, Available, Finished)

In [20]:
df_clean

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 22, Finished, Available, Finished)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,608,41,3.0,89763.84,1,0,0,199304.74,1,False,False,True,True,False
1,750,34,9.0,112822.26,1,0,0,150401.53,1,False,False,True,True,False
2,538,49,9.0,141434.04,1,0,0,173779.25,1,False,False,True,True,False
3,553,47,3.0,116528.15,1,0,0,145704.19,1,False,False,True,False,True
4,661,42,2.0,178820.91,1,0,0,29358.57,1,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,712,34,1.0,0.00,2,1,1,195052.12,0,True,False,False,False,True
9994,421,34,2.0,0.00,2,1,1,96615.23,0,True,False,False,False,True
9995,630,34,9.0,0.00,2,1,1,114006.35,0,True,False,False,False,True
9996,680,34,9.0,0.00,2,1,1,95686.60,0,True,False,False,False,True


## Model training and tracking


In [21]:
import mlflow
EXPERIMENT_NAME = "customer-experiment"  # MLflow experiment name
mlflow.set_experiment(EXPERIMENT_NAME) # Use date stamp to append to experiment
mlflow.autolog(exclusive=False)

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 23, Finished, Available, Finished)

### Prepare training and test datasets

In [22]:
y = df_clean["Exited"]
X = df_clean.drop("Exited",axis=1)
# Train-Test Separation
X_res, X_test, y_res, y_test = train_test_split(X, y, test_size=0.20, random_state=12345)

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 24, Finished, Available, Finished)

### Model training

Train the model using Random Forest with maximum depth of four, with four features.

In [23]:
mlflow.sklearn.autolog(registered_model_name='rfc1_sm')  # Register the trained model with autologging
rfc1_sm = RandomForestClassifier(max_depth=4, max_features=4, min_samples_split=3, random_state=1) # Pass hyperparameters
with mlflow.start_run(run_name="rfc1_sm") as run:
    rfc1_sm_run_id = run.info.run_id # Capture run_id for model prediction later
    print("run_id: {}; status: {}".format(rfc1_sm_run_id, run.info.status))
    # rfc1.fit(X_train,y_train) # imbalanaced training data
    rfc1_sm.fit(X_res, y_res.ravel()) # balanced training data
    rfc1_sm.score(X_test, y_test)
    y_pred = rfc1_sm.predict(X_test)
    cr_rfc1_sm = classification_report(y_test, y_pred)
    cm_rfc1_sm = confusion_matrix(y_test, y_pred)
    roc_auc_rfc1_sm = roc_auc_score(y_res, rfc1_sm.predict_proba(X_res)[:, 1])

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 25, Finished, Available, Finished)

2024/12/13 19:13:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2024/12/13 19:13:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/trusted-serv

run_id: 98adb795-b8cd-4c96-ab52-52ab46561873; status: RUNNING


Train the model using Random Forest with maximum depth of eight, with six features.

In [24]:
mlflow.sklearn.autolog(registered_model_name='rfc2_sm')  # Register the trained model with autologging
rfc2_sm = RandomForestClassifier(max_depth=8, max_features=6, min_samples_split=3, random_state=1) # Pass hyperparameters
with mlflow.start_run(run_name="rfc2_sm") as run:
    rfc2_sm_run_id = run.info.run_id # Capture run_id for model prediction later
    print("run_id: {}; status: {}".format(rfc2_sm_run_id, run.info.status))
    # rfc2.fit(X_train,y_train) # imbalanced training data
    rfc2_sm.fit(X_res, y_res.ravel()) # balanced training data
    rfc2_sm.score(X_test, y_test)
    y_pred = rfc2_sm.predict(X_test)
    cr_rfc2_sm = classification_report(y_test, y_pred)
    cm_rfc2_sm = confusion_matrix(y_test, y_pred)
    roc_auc_rfc2_sm = roc_auc_score(y_res, rfc2_sm.predict_proba(X_res)[:, 1])

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 26, Finished, Available, Finished)

run_id: 4c8f18fa-1388-4d63-9a92-0d7bbbec8c85; status: RUNNING


2024/12/13 19:14:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
Successfully registered model 'rfc2_sm'.
2024/12/13 19:14:20 INFO mlflow.store.model_registry.abstract_store: Waiting up 

## Evaluate and save the final machine learning model


Open the saved experiment from the workspace to select and save the best model.

In [25]:
# Define run_uri to fetch the model
# mlflow client: mlflow.model.url, list model
load_model_rfc1_sm = mlflow.sklearn.load_model(f"runs:/{rfc1_sm_run_id}/model")
load_model_rfc2_sm = mlflow.sklearn.load_model(f"runs:/{rfc2_sm_run_id}/model")
ypred_rfc1_sm = load_model_rfc1_sm.predict(X_test) # Random Forest with max depth of 4 and 4 features
ypred_rfc2_sm = load_model_rfc2_sm.predict(X_test) # Random Forest with max depth of 8 and 6 features

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 27, Finished, Available, Finished)

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 28, Finished, Available, Finished)

StatementMeta(, feb3e019-ec82-4891-b741-f672efa1e7ca, 29, Finished, Available, Finished)